In [1]:
import ee
import geopandas as gpd
import folium
import os
import subprocess
from shapely.geometry import mapping, MultiPolygon
from google.colab import drive
drive.mount('/content/drive')
ee.Authenticate()
# Initialize Earth Engine
ee.Initialize(project="sublime-seat-428722-n5")
#sublime-seat-428722-n5
#agrisat-463314

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ⭐ Visualisation ▶ 2 Céllules suivantes

In [3]:
# Load your shapefile
shapefile_path = f"/content/drive/MyDrive/Agriculture 2.0/Domaine Chahbi/D55/D55 Beldia1/D55 Beldia1.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg="4326")
geom = gdf.geometry.union_all()
if geom.geom_type == 'Polygon':
    geom = MultiPolygon([geom])
aoi_geom = mapping(geom)
aoi = ee.Geometry(aoi_geom)
#####################################################
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                  .filterBounds(aoi)\
                  .filterDate('2025-08-27', '2025-09-08')\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',(20)))
tiles = collection.toList(collection.size())
#####################################################
n_images = collection.size().getInfo()
images = collection.toList(n_images)
dates = []
for i in range(n_images):
    image = ee.Image(images.get(i))
    timestamp = image.date().format('YYYY-MM-dd').getInfo()
    dates.append(timestamp)
print("Acquisition dates:", dates)
#####################################################

Acquisition dates: ['2025-08-28', '2025-09-02', '2025-09-07']


# **Une seule date**

In [ ]:
import geemap

# Define color palettes for indices
index_palettes = {
    'NDVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'PRI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green'],
    'NDRE': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'GCI': ['black', 'brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MNDWI': ['brown', 'grey', 'white', 'lightblue', 'blue', 'darkblue'],
    'MVI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSI': ['darkgreen', 'green', 'lightgreen', 'yellow', 'orange','red'],
    'NDMI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
}


d = 0 # Changed from 3 to 0 to be within the valid range of indices (0, 1, 2) for the 'dates' list.
img = ee.Image(tiles.get(d))

# Sélections de bandes
red = img.select('B4')
green = img.select('B3')
blue = img.select('B2')
nir = img.select('B8')
red_edge = img.select('B5')  # ~705 nm
r531 = img.select('B5')      # Approx. 531 nm
r570 = img.select('B6')      # Approx. 570 nm
swir = img.select('B11')     # SWIR for MNDWI

# Indices standards
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI').clip(aoi)
pri = r531.subtract(r570).divide(r531.add(r570)).rename('PRI')
ndre = nir.subtract(red_edge).divide(nir.add(red_edge)).rename('NDRE')
gci = (nir.divide(green)).subtract(1).rename('GCI')
L = 0.5
mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')


# MVI: Modified Vegetation Index
eps = ee.Number(1e-6)  # avoid divide-by-zero
ratio = nir.subtract(red).divide(nir.add(red).add(eps))
mvi = ratio.add(0.5).sqrt().rename('MVI') # Added .rename('MVI')

ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
msi = swir.divide(nir).rename('MSI')
# GDD: Exemple générique (remplace `tempImg` par une vraie image de température en °C)
# GDD = Tmoy - 10 (base 10°C pour la croissance végétale)
# tempImg = ee.Image(...)  # Remplace ceci par l’image de température
# gdd = tempImg.subtract(10).max(0).rename('GDD')

# Liste des indices à exporter
ls = [ndvi, pri, ndre, gci, mndwi, mvi,ndmi,msi]  # gdd à ajouter si disponible
ls_str = ["NDVI", "PRI", "NDRE", "GCI", "MNDWI", "MVI","NDMI","MSI"]  # Ajouter "GDD" si utilisé
###########################################################################

# Initialize the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=18)
Map.add_basemap("SATELLITE")

# Add each index with dynamic min/max and stretched palette
for ind, ind_st in zip(ls, ls_str):
    # Compute min/max dynamically
    stats = ind.reduceRegion(
        reducer=ee.Reducer.percentile([2, 98]),  # more robust than minMax
        geometry=aoi,
        scale=10,
        maxPixels=1e13
    )

    try:
        min_val = stats.getNumber(f"{ind_st}_p2").getInfo()
        max_val = stats.getNumber(f"{ind_st}_p98").getInfo()
    except Exception:
        print(f"⚠️ Warning: Could not compute min/max for {ind_st}, skipping layer.")
        continue

    vis_params = {
        'min': min_val,
        'max': max_val,
        'palette': index_palettes[ind_st]
    }

    Map.addLayer(ind.clip(aoi), vis_params, f'{ind_st}')
    Map.add_colorbar(vis_params, label=f'{ind_st}', layer_name=f'{ind_st}',position='bottomleft')

# Add AOI overlay
Map.addLayer(aoi, {'color': 'red'}, 'Point of Interest')
dir_dw = f"{dates[d]}_{shapefile_path[39:53]}"
# Show the map
Map
##################################################

Map(center=[31.4419937962541, -8.574930824281235], controls=(WidgetControl(options=['position', 'transparent_b…

# **Plusieurs dates**

In [4]:
import geemap

# Define color palettes for indices
index_palettes = {
    'NDVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'PRI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green'],
    'NDRE': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'GCI': ['black', 'brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MNDWI': ['brown', 'grey', 'white', 'lightblue', 'blue', 'darkblue'],
    'MVI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSI': ['darkgreen', 'green', 'lightgreen', 'yellow', 'orange','red'],
    'NDMI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
}

# Initialize the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=18)
Map.add_basemap("SATELLITE")
Map.addLayer(aoi, {'color': 'red'}, 'Point of Interest')

# ---- Boucle sur toutes les dates ----
for d in range(len(dates)):
    img = ee.Image(tiles.get(d))

    # Sélections de bandes
    red = img.select('B4')
    green = img.select('B3')
    blue = img.select('B2')
    nir = img.select('B8')
    red_edge = img.select('B5')  # ~705 nm
    r531 = img.select('B5')      # Approx. 531 nm
    r570 = img.select('B6')      # Approx. 570 nm
    swir = img.select('B11')     # SWIR for MNDWI

    # Indices standards
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI').clip(aoi)
    pri = r531.subtract(r570).divide(r531.add(r570)).rename('PRI')
    ndre = nir.subtract(red_edge).divide(nir.add(red_edge)).rename('NDRE')
    gci = (nir.divide(green)).subtract(1).rename('GCI')
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')

    # MVI: Modified Vegetation Index
    eps = ee.Number(1e-6)
    ratio = nir.subtract(red).divide(nir.add(red).add(eps))
    mvi = ratio.add(0.5).sqrt().rename('MVI')

    ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
    msi = swir.divide(nir).rename('MSI')

    ls = [ndvi, pri, ndre, gci, mndwi, mvi, ndmi, msi]
    ls_str = ["NDVI", "PRI", "NDRE", "GCI", "MNDWI", "MVI", "NDMI", "MSI"]

    for ind, ind_st in zip(ls, ls_str):
        # Compute min/max dynamically
        stats = ind.reduceRegion(
            reducer=ee.Reducer.percentile([2, 98]),
            geometry=aoi,
            scale=10,
            maxPixels=1e13
        )
        try:
            min_val = stats.getNumber(f"{ind_st}_p2").getInfo()
            max_val = stats.getNumber(f"{ind_st}_p98").getInfo()
        except Exception:
            print(f"⚠️ Warning: Could not compute min/max for {ind_st} on {dates[d]}, skipping layer.")
            continue

        vis_params = {
            'min': min_val,
            'max': max_val,
            'palette': index_palettes[ind_st]
        }

        layer_name = f"{ind_st} - {dates[d]}"
        Map.addLayer(ind.clip(aoi), vis_params, layer_name)
        Map.add_colorbar(vis_params, label=f"{ind_st}", layer_name=layer_name, position='bottomleft')

# Afficher la carte
Map


Map(center=[31.4419937962541, -8.574930824281235], controls=(WidgetControl(options=['position', 'transparent_b…

# **TELECHARGER LES HEATMAPS EN PNG**

In [15]:
import ee
import geemap
from PIL import Image, ImageDraw, ImageFont
import requests
import os

# ---- Monter Google Drive ----
from google.colab import drive
drive.mount('/content/drive')

# ---- Dossier de sortie ----
drive_dir = "/content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports"
os.makedirs(drive_dir, exist_ok=True)

# ---- Palettes de couleurs ----
palettes = {
    'NDVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'PRI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green'],
    'NDRE': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'GCI': ['black', 'brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MNDWI': ['brown', 'grey', 'white', 'lightblue', 'blue', 'darkblue'],
    'MVI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSI': ['darkgreen', 'green', 'lightgreen', 'yellow', 'orange','red'],
    'NDMI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
}

# ---- Polices (fallback si indisponible) ----
try:
    font_title = ImageFont.truetype("DejaVuSans-Bold.ttf", 72)
    font_minmax = ImageFont.truetype("DejaVuSans.ttf", 24)
except:
    font_title = ImageFont.load_default()
    font_minmax = ImageFont.load_default()

# ---- Boucle sur toutes les dates ----
for d, date_str in enumerate(dates):
    print(f"\n📅 Traitement des images du {date_str} ...")
    img_ee = ee.Image(tiles.get(d))

    # ---- Recalculer les indices pour cette date ----
    red = img_ee.select('B4')
    green = img_ee.select('B3')
    blue = img_ee.select('B2')
    nir = img_ee.select('B8')
    red_edge = img_ee.select('B5')
    r531 = img_ee.select('B5')
    r570 = img_ee.select('B6')
    swir = img_ee.select('B11')

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI').clip(aoi)
    pri = r531.subtract(r570).divide(r531.add(r570)).rename('PRI').clip(aoi)
    ndre = nir.subtract(red_edge).divide(nir.add(red_edge)).rename('NDRE').clip(aoi)
    gci = (nir.divide(green)).subtract(1).rename('GCI').clip(aoi)
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI').clip(aoi)
    eps = ee.Number(1e-6)
    ratio = nir.subtract(red).divide(nir.add(red).add(eps))
    mvi = ratio.add(0.5).sqrt().rename('MVI').clip(aoi)
    ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI').clip(aoi)
    msi = swir.divide(nir).rename('MSI').clip(aoi)

    indices_calc = {
        "NDVI": ndvi,
        "PRI": pri,
        "NDRE": ndre,
        "GCI": gci,
        "MNDWI": mndwi,
        "MVI": mvi,
        "NDMI": ndmi,
        "MSI": msi
    }

    # ---- Export chaque indice ----
    for ind_name, ind_img in indices_calc.items():
        palette = palettes[ind_name]

        # Stats pour min/max
        stats = ind_img.reduceRegion(
            reducer=ee.Reducer.percentile([2,98]),
            geometry=aoi,
            scale=10,
            maxPixels=1e13
        )
        try:
            min_val = stats.getNumber(f"{ind_name}_p2").getInfo()
            max_val = stats.getNumber(f"{ind_name}_p98").getInfo()
        except:
            print(f"⚠️ Impossible de calculer min/max pour {ind_name} ({date_str})")
            continue

        vis_params = {"min": min_val, "max": max_val, "palette": palette}

        # ---- Télécharger via URL Thumb (dimensions homogènes) ----
        url = ind_img.visualize(**vis_params).getThumbURL({
            'region': aoi,
            'dimensions': 1024,
            'format': 'png'
        })

        out_path = os.path.join(drive_dir, f"{ind_name}_{date_str}_raw.png")
        r = requests.get(url)
        with open(out_path, "wb") as f:
            f.write(r.content)

        # ---- Ajouter marge, titre et légende ----
        img = Image.open(out_path)
        w, h = img.size
        margin = int(0.15 * max(w, h))
        rect_h = 80
        new_w, new_h = w + 2*margin, h + 2*margin + rect_h + 20
        img_with_margin = Image.new('RGB', (new_w, new_h), (255, 255, 255))
        img_with_margin.paste(img, (margin, margin))
        draw = ImageDraw.Draw(img_with_margin)

        # Rectangle min/max
        rect_y = new_h - rect_h - 10
        rect_x1, rect_x2 = margin, new_w - margin
        for i in range(rect_x2 - rect_x1):
            ratio = i / (rect_x2 - rect_x1)
            idx = int(ratio * (len(palette)-1))
            color = palette[idx]
            draw.line([(rect_x1+i, rect_y), (rect_x1+i, rect_y+rect_h)], fill=color)

        draw.text((rect_x1+10, rect_y+rect_h//4), f"Min: {min_val:.2f}", fill="black", font=font_minmax)
        draw.text((rect_x2-200, rect_y+rect_h//4), f"Max: {max_val:.2f}", fill="black", font=font_minmax)

        # Titre
        title_text = f"{ind_name} - {date_str}"
        bbox = draw.textbbox((0, 0), title_text, font=font_title)
        text_w = bbox[2] - bbox[0]
        title_x = (new_w - text_w) // 2
        draw.text((title_x, 10), title_text, fill="black", font=font_title)

        # Sauvegarder PNG final
        png_path = os.path.join(drive_dir, f"{ind_name}_{date_str}_margin.png")
        img_with_margin.save(png_path)
        print(f"✅ {ind_name} {date_str} -> {png_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📅 Traitement des images du 2025-08-28 ...
✅ NDVI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/NDVI_2025-08-28_margin.png
✅ PRI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/PRI_2025-08-28_margin.png
✅ NDRE 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/NDRE_2025-08-28_margin.png
✅ GCI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/GCI_2025-08-28_margin.png
✅ MNDWI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/MNDWI_2025-08-28_margin.png
✅ MVI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/MVI_2025-08-28_margin.png
✅ NDMI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/NDMI_2025-08-28_margin.png
✅ MSI 2025-08-28 -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/MSI_2025-08-28_margin.png

📅

# **REGROUPER LES IMAGES**

In [16]:
from PIL import Image, ImageDraw, ImageFont
import os
from datetime import datetime

# ---- Dossier source PNG marginés ----
drive_dir = "/content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports"

# ---- Dossier de sortie pour les montages ----
composite_dir = "/content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images"
os.makedirs(composite_dir, exist_ok=True)

# ---- Liste des indices présents dans les fichiers ----
indices = set(f.split("_")[0] for f in os.listdir(drive_dir) if f.endswith("_margin.png"))

# ---- Fonction pour extraire la date depuis le nom de fichier ----
def extract_date(filename):
    # Exemple : NDVI_2025-08-28_margin.png
    parts = filename.split("_")
    date_str = parts[1]  # "2025-08-28"
    return datetime.strptime(date_str, "%Y-%m-%d")

# ---- Boucle sur chaque indice ----
for indice in indices:
    # Liste des PNG du même indice
    png_files = [f for f in os.listdir(drive_dir) if f.startswith(indice) and f.endswith("_margin.png")]

    if not png_files:
        continue

    # Trier par date croissante
    png_files.sort(key=extract_date)

    # On prend jusqu'à 4 dernières images pour montage 2x2
    png_files = png_files[-4:]

    images = [Image.open(os.path.join(drive_dir, f)) for f in png_files]

    # ---- Trouver la taille max des images pour montage ----
    max_w = max(img.width for img in images)
    max_h = max(img.height for img in images)

    # ---- Taille finale montage 2x2 ----
    cols, rows = 2, 2
    montage_w = cols * max_w
    montage_h = rows * max_h
    montage_img = Image.new('RGB', (montage_w, montage_h), (255,255,255))

    # ---- Font pour titre de la date ----
    try:
        font_date = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 40)
    except OSError:
        font_date = ImageFont.load_default()

    # ---- Coller les images et dessiner la date ----
    for idx, img in enumerate(images):
        row = idx // cols
        col = idx % cols
        x = col * max_w
        y = row * max_h
        montage_img.paste(img, (x, y))

        # Dessiner le titre de la date
        draw = ImageDraw.Draw(montage_img)
        date_text = extract_date(png_files[idx]).strftime("%d-%m-%Y")
        bbox = draw.textbbox((0,0), date_text, font=font_date)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
        text_x = x + (img.width - text_w)//2
        text_y = y + 10  # 10 px du haut de l'image
        draw.text((text_x, text_y), date_text, fill="black", font=font_date)

    # ---- Sauvegarder montage final ----
    montage_path = os.path.join(composite_dir, f"{indice}_montage_2x2.png")
    montage_img.save(montage_path)
    print(f"✅ Montage 2x2 pour {indice} -> {montage_path}")


✅ Montage 2x2 pour NDVI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/NDVI_montage_2x2.png
✅ Montage 2x2 pour PRI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/PRI_montage_2x2.png
✅ Montage 2x2 pour GCI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/GCI_montage_2x2.png
✅ Montage 2x2 pour MNDWI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/MNDWI_montage_2x2.png
✅ Montage 2x2 pour NDRE -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/NDRE_montage_2x2.png
✅ Montage 2x2 pour MSI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/MSI_montage_2x2.png
✅ Montage 2x2 pour NDMI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/NDMI_montage_2x2.png
✅ Montage 2x2 pour MVI -> /content/drive/MyDrive/Agriculture 2.0/Heatmaps_Exports/Composite_images/MVI_montage_2x2.png


# 🔼 Téléchargement

In [ ]:
dir_dw = f"{dates[d]}_{shapefile_path[39:53]}"
#os.mkdir(f"./{dir_dw}")
for ind, ind_st in zip(ls, ls_str):
    ee.batch.Export.image.toDrive(
        image=ind.clip(aoi),
        description=f'{ind_st}_{dates[d]}',
        folder=dir_dw,
        #fileNamePrefix=f'{ind_st}_{dates[d]}',
        scale=10,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    ).start()

⚠️ A exécuter aprés le téléchargement des fichiers




In [ ]:
dir = f"{dates[d]}_{shapefile_path[39:53]}"
tifs = [f"{dir}/{tif}" for tif in os.listdir(dir) if tif.lower().endswith('.tif')]
for tif in tifs:
    print(f"Setting NoData = 'nan' for {tif}")
    subprocess.run(['gdal_edit.py', '-a_nodata', 'nan', tif])
print("✅ All files updated with NoData = nan.")

FileNotFoundError: [Errno 2] No such file or directory: './2025-08-20_AOI Benyagrine'

In [ ]:
# Load your shapefile
shapefile_path = f"/content/drive/MyDrive/Agriculture 2.0/AOIP1/AOIP1.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg="4326")
geom = gdf.geometry.union_all()

# Ensure it's a MultiPolygon even if it’s a single polygon
if geom.geom_type == 'Polygon':
    geom = MultiPolygon([geom])

# Convert to GeoJSON format
aoi_geom = mapping(geom)

# Convert to Earth Engine geometry
aoi = ee.Geometry(aoi_geom)

In [ ]:
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
                  .filterBounds(aoi)\
                  .filterDate('2025-08-04', '2025-08-30')\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',(10)))
tiles = collection.toList(collection.size())

In [ ]:
n_images = collection.size().getInfo()
images = collection.toList(n_images)

# Loop and extract dates
dates = []
for i in range(n_images):
    image = ee.Image(images.get(i))
    timestamp = image.date().format('YYYY-MM-dd').getInfo()
    dates.append(timestamp)
print("Acquisition dates:", dates)

Acquisition dates: ['2025-08-08', '2025-08-13', '2025-08-20']


In [ ]:
d = 2 # Changed from 3 to 0 to be within the valid range of indices (0, 1, 2) for the 'dates' list.
img = ee.Image(tiles.get(d))

# Sélections de bandes
red = img.select('B4')
green = img.select('B3')
blue = img.select('B2')
nir = img.select('B8')
red_edge = img.select('B5')  # ~705 nm
r531 = img.select('B5')      # Approx. 531 nm
r570 = img.select('B6')      # Approx. 570 nm
swir = img.select('B11')     # SWIR for MNDWI

# Indices standards
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI').clip(aoi)
pri = r531.subtract(r570).divide(r531.add(r570)).rename('PRI')
ndre = nir.subtract(red_edge).divide(nir.add(red_edge)).rename('NDRE')
gci = (nir.divide(green)).subtract(1).rename('GCI')
L = 0.5
savi = nir.subtract(red).multiply(1 + L).divide(nir.add(red).add(L)).rename('SAVI')
osavi = nir.subtract(red).divide(nir.add(red).add(0.16)).rename('OSAVI')
msavi2 = nir.multiply(2).add(1).subtract(
    nir.multiply(2).add(1).pow(2).subtract(nir.subtract(red).multiply(8)).sqrt()
).divide(2).rename('MSAVI2')
mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')

# Indices ajoutés
# TCARI: Transformed Chlorophyll Absorption in Reflectance Index
tcari = ee.Image(3).multiply(
    red_edge.subtract(red).subtract(
        red_edge.subtract(green).multiply(0.2).multiply(red_edge.divide(red))
    )
).rename('TCARI')

# MCARI: Modified Chlorophyll Absorption in Reflectance Index
#mcari = red_edge.subtract(red).subtract((red_edge.subtract(green)).multiply(0.2)).multiply(red_edge.divide(red)).rename('MCARI')
mcari = red_edge.subtract(red).subtract(
    red_edge.subtract(green).multiply(0.2)
).multiply(red_edge.divide(green)).rename('MCARI')

# ClGreen: Chlorophyll Green Index (alternative form of GCI)
clgreen = nir.divide(green).rename('CLGREEN')

# MVI: Modified Vegetation Index
eps = ee.Number(1e-6)  # avoid divide-by-zero
ratio = nir.subtract(red).divide(nir.add(red).add(eps))
mvi = ratio.add(0.5).sqrt()

ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
msi = swir.divide(nir).rename('MSI')
# GDD: Exemple générique (remplace `tempImg` par une vraie image de température en °C)
# GDD = Tmoy - 10 (base 10°C pour la croissance végétale)
# tempImg = ee.Image(...)  # Remplace ceci par l’image de température
# gdd = tempImg.subtract(10).max(0).rename('GDD')

# Liste des indices à exporter
ls = [ndvi, pri, ndre, gci, savi, osavi, msavi2, mndwi, tcari, mcari, clgreen, mvi,ndmi,msi]  # gdd à ajouter si disponible
ls_str = ["NDVI", "PRI", "NDRE", "GCI", "SAVI", "OSAVI", "MSAVI2", "MNDWI", "TCARI", "MCARI", 'CLGREEN', "MVI","NDMI","MSI"]  # Ajouter "GDD" si utilisé

dir_dw = f"{dates[d]}_{shapefile_path[39:53]}"
#os.mkdir(f"./{dir_dw}")
for ind, ind_st in zip(ls, ls_str):
    ee.batch.Export.image.toDrive(
        image=ind.clip(aoi),
        description=f'{ind_st}_{dates[d]}',
        folder=dir_dw,
        #fileNamePrefix=f'{ind_st}_{dates[d]}',
        scale=10,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    ).start()

# ⚠️ A exécuter aprés le téléchargement des fichiers




In [ ]:
dir = f"./{dates[d]}_{shapefile_path[39:53]}"
tifs = [f"{dir}/{tif}" for tif in os.listdir(dir) if tif.lower().endswith('.tif')]
for tif in tifs:
    print(f"Setting NoData = 'nan' for {tif}")
    subprocess.run(['gdal_edit.py', '-a_nodata', 'nan', tif])
print("✅ All files updated with NoData = nan.")

FileNotFoundError: [Errno 2] No such file or directory: './2025-08-20_AOI Benyagrine'

# Visualisation

In [ ]:
import geemap

index_palettes = {
    'NDVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'PRI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green'],
    'NDRE': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'GCI': ['black', 'brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'SAVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'OSAVI': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSAVI2': ['brown', 'red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MNDWI': ['brown', 'grey', 'white', 'lightblue', 'blue', 'darkblue'],
    'TCARI': ['red', 'orange', 'yellow', 'white', 'lightgreen', 'green'],
    'MCARI': ['red', 'orange', 'yellow', 'white', 'lightgreen', 'green'],
    'CLGREEN': ['black', 'brown', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MVI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen'],
    'MSI': ['darkgreen', 'green', 'lightgreen', 'yellow', 'orange','red'],
    'NDMI': ['red', 'orange', 'yellow', 'lightgreen', 'green', 'darkgreen']
}
# Initialize the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=18)

# Add each index with dynamic min/max and stretched palette
for ind, ind_st in zip(ls, ls_str):
    # Compute min/max dynamically
    stats = ind.reduceRegion(
        reducer=ee.Reducer.percentile([2, 98]),  # more robust than minMax
        geometry=aoi,
        scale=10,
        maxPixels=1e13
    )

    try:
        min_val = stats.getNumber(f"{ind_st}_p2").getInfo()
        max_val = stats.getNumber(f"{ind_st}_p98").getInfo()
    except Exception:
        print(f"⚠️ Warning: Could not compute min/max for {ind_st}, skipping layer.")
        continue

    vis_params = {
        'min': min_val,
        'max': max_val,
        'palette': index_palettes[ind_st]
    }

    Map.addLayer(ind.clip(aoi), vis_params, f'{ind_st}')
    Map.add_colorbar(vis_params, label=f'{ind_st}', layer_name=f'{ind_st}',position='bottomleft')

# Add AOI overlay
Map.addLayer(aoi, {'color': 'red'}, 'Point of Interest')

# Show the map
Map

⚠️ Warning: Could not compute min/max for MVI, skipping layer.


Map(center=[31.438628597724573, -8.573076195405497], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
rgb = img.select(['B4', 'B3', 'B2']).rename(['R', 'G', 'B'])
task = ee.batch.Export.image.toDrive(
    image=rgb,
    description='RGB_Export',
    folder='Imad/20',
    fileNamePrefix='RGB_Composite_06202020',
    region=aoi,
    scale=10,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()